Setup
=
The function below formats a monetary amount, something which comes up lot later.

In [ ]:
def euro(number):
    return "€%.02f" % number

print(euro(123.456))

A class representing a person. It's essentially a tuple, there is no behaviour attached to it. 

In [ ]:
class Person:
    def __init__(self, name):
        self.balance = 0
        self.name = str(name)
        self.kitty = None
        
kerry = Person("Kerry")
print(kerry.name)
print(kerry.balance)

A "Transaction" object stores information about a single transaction - the payer, the amount paid and a descriptive text item.

In [ ]:
class Transaction:
    def __init__(self, payer, amount, desc):
        self.payer = payer
        self.amount = amount
        self.desc = desc

The Kitty object contains most of the logic for this application.

The class defines three main functions:

- **add_transaction**

This will apply a new transaction to the kitty. This means appending a log entry and also changing each person's balance accordingly.
    
   
 - **print_status**


Prints the current total, historical information and current balances recorded by the Kitty.


 - **reconcile**

Calculates a sequence of payments that will resolve debts between the members of the Kitty, and ends the event.

In [ ]:
class Kitty:
    
    def __init__(self, people, desc):
        
        # Note: add_transaction is robust against invalid data, 
        # as specified in the assignment description. This constructor 
        # however is not - anything could be passed in as people.
        self.start(people, desc)
    
    def start(self, people, desc):
        
        self.__people = list(people)
        self.__desc = str(desc)
        self.__log = []
        self.__total = 0
        self.__active = True
        
        for person in self.__people:
            
            if person.kitty is not None:
                raise ValueError("A person may not be registered" + 
                        " to more than one event at a time")
            
            person.kitty = self
        
    def end(self):
        
        for person in self.__people:
            person.kitty = None
        
        self.__people = None
        self.__desc = None
        self.__log = None
        self.__total = 0
        self.__active = False

    
    def add_transaction(self, payer, amount, desc):
        
        if not self.__active: 
            raise RuntimeError("No event is in progress")

        if payer not in self.__people: 
            raise ValueError("Payer must be registered to the event")
        
        try: 
            amount = float(amount)
        except (ValueError, TypeError) as e:
            raise ValueError("Failed to parse amount as float")  
        
        if amount <= 0:
            raise ValueError("Amount paid by the payer must be positive")
        
        # The log is for printing historial info only
        self.__log.append(Transaction(payer, amount, str(desc)))
        
        # The balances and total are used in reconciliation calculation
        payer.balance -= amount
        for person in self.__people:
            person.balance += amount/len(self.__people)
        self.__total += amount
        
    def print_status(self):
        
        if not self.__active:
            print("No event in progress")
            return
        
        print("Kitty:", self.__desc)
        
        total = self.__total
        each = total/len(self.__people)
        
        print(euro(total), ": ", euro(each), " each", sep="")
        
        print()
        
        for t in self.__log:
            print(t.payer.name, "paid", euro(t.amount), "for", t.desc)
        
        print()
        
        for person in self.__people:
            print(person.name, end=" ")
            print("has balance", euro(person.balance))
            
    def reconcile(self):
        
        if not self.__active:
            print("No event in progress")
            return
        
        # All balances *should* sum to zero.
        # To reconcile, those with positive balance
        # should pay those with negative balance
        
        debtors = []
        creditors = []
        
        for person in self.__people:
            if person.balance > 0:
                debtors.append(person)
            elif person.balance < 0:
                creditors.append(person)
        
        # Iterate through the creditors and debtors in tandem.
        
        # The current debtor pays until they owe no more,
        # then the next debtor is chosen. The current creditor recieves
        # until they are owed no more, then the next creditor is chosen
        
        i = j = 0
        # In theory this should be an "or", but an "and" seems safer where floating point
        # errors are concerned.
        while i < len(debtors) and j < len(creditors):

            debtor = debtors[i]
            creditor = creditors[j]
            
            amount = min(debtor.balance, abs(creditor.balance))
            
            print(debtor.name, "pays", creditor.name, euro(amount))
            
            debtor.balance -= amount
            creditor.balance += amount
            
            if abs(debtor.balance) < 0.005:
                i += 1
            
            if abs(creditor.balance) < 0.005:
                j += 1
                
        # End the event
        self.end()


Sample 1
==

In [ ]:
annie = Person("Annie")
bill = Person("Bill")
sally = Person("Sally")

kitty = Kitty([annie, bill, sally], "Concert")

kitty.add_transaction(annie, 180, "tickets")
kitty.add_transaction(sally, 75, "dinner")
kitty.add_transaction(bill, 19, "drinks")
kitty.add_transaction(bill, 16, "the taxi")

kitty.print_status()
print()
kitty.reconcile()

Sample 2
==

In [ ]:
cathy = Person("Cathy")
robin = Person("Robin")
jen = Person("Jen")

kitty = Kitty([cathy, robin, jen], "Cinema")

kitty.add_transaction(cathy, 33, "tickets")
kitty.add_transaction(robin, 60, "dinner")
kitty.add_transaction(jen, 21, "drinks")
kitty.add_transaction(jen, 27, "the taxi")

kitty.print_status()
print()
kitty.reconcile()

Sample 3
==

In [ ]:
nora = Person("Nora")
eva = Person("Eva")
frankie = Person("Frankie")
harry = Person("Harry")

kitty = Kitty([nora, eva, frankie, harry], "Concert")

kitty.add_transaction(nora, 110, "dinner")
kitty.add_transaction(eva, 60, "lunch")
kitty.add_transaction(frankie, 125, "dinner")
kitty.add_transaction(harry, 70, "lunch")

kitty.print_status()
print()
kitty.reconcile()

Some extra tests regarding errors
--

In [ ]:
nora = Person("Nora")
eva = Person("Eva")
frankie = Person("Frankie")

kitty = Kitty([nora, eva, frankie], "Concert")

kitty.add_transaction(eva, 150, "tickets")

A person can only take part in one event at a time

In [ ]:
second_kitty = Kitty([nora, eva], "Cinema")

Transactions should only be added if the member is part of the event:

In [ ]:
jor = Person("Jor")
kitty.add_transaction(jor, 70, "lunch")

Amount should be a number:

In [ ]:
kitty.add_transaction(eva, None, "lunch")

In [ ]:
kitty.add_transaction(eva, "Hello World", "lunch")

Amount should be positive:

In [ ]:
kitty.add_transaction(eva, -23, "lunch")

In [ ]:
kitty.add_transaction(eva, 0, "lunch")

Once an event is reconciled, it should not be possible to add a transaction until a new event is started:

In [ ]:
kitty.reconcile()
kitty.add_transaction(eva, 23, "lunch")

No event should be in progress after reconciliation:

In [ ]:
kitty.print_status()

In [ ]:
kitty.reconcile() 